In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from pathlib import Path

In [2]:
db_path = Path("test.db")

if db_path.exists():
    db_path.unlink()

In [3]:
engine = create_engine("sqlite:///test.db")

In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE IF NOT EXISTS test (a INT, b TEXT)"))

In [5]:
with engine.connect() as conn:
    conn.execute(text("INSERT INTO test (a, b) VALUES (1, 'foo')"))
    conn.execute(text("INSERT INTO test (a, b) VALUES (2, 'bar')"))
    conn.execute(text("INSERT INTO test (a, b) VALUES (3, 'baz')"))
    conn.commit()

In [6]:
df = pd.read_sql_query("SELECT * FROM test", engine)

df

,a,b
0,1,foo
1,2,bar
2,3,baz


In [7]:
# add new column to dataframe

df["c"] = None
df["d"] = None

# insert new row
df.loc[3] = [4, "qux", "lorem", "ipsum"]
df.loc[4] = [5, "quux", "dolor", "sit"]
df.loc[5] = [6, "corge", "amet", "consectetur"]

# drop old rows
df = df.drop([0, 1, 2])

df

,a,b,c,d
3,4,qux,lorem,ipsum
4,5,quux,dolor,sit
5,6,corge,amet,consectetur


In [8]:
from sqlmigrately import migrate_table

In [9]:
migrate_table(
    "test",
    df,
    engine,
    push_data=True,
    # column_type_map={"c": "TEXT", "d": "INTEGER"},
)

2024-11-23 21:30:46.539 | INFO     | sqlmigrately.main:migrate_table:45 - Detected new columns: [{'name': 'c', 'type': dtype('O')}, {'name': 'd', 'type': dtype('O')}]
2024-11-23 21:30:46.540 | INFO     | sqlmigrately.utils:alter_table:103 - Executing: ALTER TABLE test ADD COLUMN c TEXT
2024-11-23 21:30:46.562 | INFO     | sqlmigrately.utils:alter_table:103 - Executing: ALTER TABLE test ADD COLUMN d TEXT
2024-11-23 21:30:46.576 | INFO     | sqlmigrately.main:migrate_table:55 - Appending data to table: test


In [10]:
new_table = pd.read_sql_query("SELECT * FROM test", engine)

new_table

,a,b,c,d
0,1,foo,None,None
1,2,bar,None,None
2,3,baz,None,None
3,4,qux,lorem,ipsum
4,5,quux,dolor,sit
5,6,corge,amet,consectetur


In [11]:
# delete column from dataframe

del new_table["a"]

new_table = new_table.drop([0, 1, 2, 3, 4, 5])

new_table.loc[0] = ["test", "test2", "deez"]
new_table.loc[1] = ["test3", "test4", "nutz"]

new_table

,b,c,d
0,test,test2,deez
1,test3,test4,nutz


In [12]:
migrate_table("test", new_table, engine, push_data=True, remove_cols=True)

2024-11-23 21:30:46.646 | INFO     | sqlmigrately.main:migrate_table:50 - Detected removed columns: [{'name': 'a', 'type': INTEGER()}]
2024-11-23 21:30:46.647 | INFO     | sqlmigrately.utils:alter_table:103 - Executing: ALTER TABLE test DROP COLUMN a
2024-11-23 21:30:46.668 | INFO     | sqlmigrately.main:migrate_table:55 - Appending data to table: test


In [13]:
final_table = pd.read_sql_query("SELECT * FROM test", engine)

final_table

,b,c,d
0,foo,None,None
1,bar,None,None
2,baz,None,None
3,qux,lorem,ipsum
4,quux,dolor,sit
5,corge,amet,consectetur
6,test,test2,deez
7,test3,test4,nutz


In [14]:
# test exceptions
from sqlmigrately.exceptions import TableDoesNotExistError

try:
    migrate_table("test2", new_table, engine)
except TableDoesNotExistError as e:
    print(e)

Table test2 does not exist


In [15]:
# delete database
db_path.unlink()